In [29]:
import simpy
import numpy as np

class Server:
    def __init__(self, env):
        self.env = env
        self.machine = simpy.Resource(env, capacity=1)
        self.latest_task_end_time = 0

    def process(self, task):
        with self.machine.request() as req:
            yield req
            yield self.env.timeout(task.duration)
            self.latest_task_end_time = self.env.now

class Task:
    def __init__(self, duration):
        self.duration = duration

class FitnessFunction:
    def __init__(self, tasks):
        self.tasks = tasks

    def __call__(self, solution):
        env = simpy.Environment()
        servers = [Server(env) for _ in range(10)]

        for task_duration, server_index in zip(self.tasks, solution):
            task = Task(task_duration)
            server = servers[int(server_index)]
            env.process(server.process(task))

        env.run()

        makespan = max(server.latest_task_end_time for server in servers)
        return makespan

class Flower:
    def __init__(self, decision_variables, lower_bound, upper_bound):
        self.position = np.random.uniform(low=lower_bound, high=upper_bound, size=decision_variables)
        self.fitness = float('inf')

class FlowerPollination:
    def __init__(self, fitness_function, population_size=20, decision_variables=10, lower_bound=0, upper_bound=1, generations=100, switch_prob=0.8, gamma=0.1, beta=1.5):
        self.population = [Flower(decision_variables, lower_bound, upper_bound) for _ in range(population_size)]
        self.global_best = None
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.decision_variables = decision_variables
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.generations = generations
        self.switch_prob = switch_prob
        self.gamma = gamma
        self.beta = beta

    def levy_flight(self, beta):
        sigma = (np.math.gamma(1 + beta) * np.sin(np.pi * beta / 2) / (np.math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.normal(0, sigma, size=self.decision_variables)
        v = np.random.normal(0, 1, size=self.decision_variables)
        step = u / abs(v) ** (1 / beta)
        return step

    def global_pollination(self, flower):
        step_size = self.gamma * self.levy_flight(self.beta)
        new_position = flower.position + step_size * (flower.position - self.global_best.position)
        return new_position

    def local_pollination(self, flower):
        flower_j = np.random.choice(self.population)
        epsilon = np.random.uniform(low=0, high=1, size=self.decision_variables)
        new_position = flower.position + epsilon * (flower_j.position - flower.position)
        return new_position

    def pollination(self, flower):
        r = np.random.uniform(low=0, high=1)
        if r < self.switch_prob:
            new_position = self.global_pollination(flower)
        else:
            new_position = self.local_pollination(flower)
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def find_global_best(self):
        for flower in self.population:
            if flower.fitness < self.global_best.fitness:
                self.global_best = flower

    def optimize(self):
        self.global_best = self.population[0]
        self.global_best.fitness = self.fitness_function(self.global_best.position)

        for g in range(self.generations):
            for flower in self.population:
                flower.fitness = self.fitness_function(flower.position)
                self.find_global_best()

                new_position = self.pollination(flower)
                new_fitness = self.fitness_function(new_position)

                if new_fitness < flower.fitness:
                    flower.position = new_position
                    flower.fitness = new_fitness

            self.find_global_best()
            print(f"Generation: {g}, Best fitness: {self.global_best.fitness}")

        return self.global_best

class SimulatedAnnealing:
    def __init__(self, fitness_function, initial_solution, initial_temperature, cooling_rate):
        self.fitness_function = fitness_function
        self.current_solution = initial_solution
        self.current_fitness = self.fitness_function(self.current_solution)
        self.temperature = initial_temperature
        self.cooling_rate = cooling_rate

    def neighbour(self):
        new_solution = self.current_solution.copy()
        task_index = np.random.randint(0, len(new_solution))
        new_solution[task_index] = np.random.uniform(low=0, high=9)
        return new_solution

    def accept(self, candidate_fitness):
        if candidate_fitness < self.current_fitness:
            return True
        else:
            return np.random.rand() < np.exp((self.current_fitness - candidate_fitness) / self.temperature)

    def anneal(self):
        while self.temperature > 0.001:
            candidate_solution = self.neighbour()
            candidate_fitness = self.fitness_function(candidate_solution)

            if self.accept(candidate_fitness):
                self.current_solution = candidate_solution
                self.current_fitness = candidate_fitness

            self.temperature *= self.cooling_rate
            print(f"Temperature: {self.temperature}, Current fitness: {self.current_fitness}")

        return self.current_solution

def generate_schedule(task_sizes):
    for task_size in task_sizes:
        task_durations = np.random.uniform(low=1, high=10, size=task_size)
        fitness_function = FitnessFunction(task_durations)

        fpa = FlowerPollination(fitness_function, population_size=100, decision_variables=task_size, lower_bound=0, upper_bound=9, generations=50)
        best_solution = fpa.optimize()
        print(f"Task size: {task_size}, Best makespan (FPA): {best_solution.fitness}")

        sa = SimulatedAnnealing(fitness_function, best_solution.position, initial_temperature=10, cooling_rate=0.9)
        best_solution_sa = sa.anneal()
        print(f"Task size: {task_size}, Best makespan (SA): {fitness_function(best_solution_sa)}")

task_sizes = [200]
generate_schedule(task_sizes)


Generation: 0, Best fitness: 129.20929246154336
Generation: 1, Best fitness: 129.20929246154336
Generation: 2, Best fitness: 129.20929246154336
Generation: 3, Best fitness: 129.20929246154336
Generation: 4, Best fitness: 127.96466573142229
Generation: 5, Best fitness: 127.96466573142229
Generation: 6, Best fitness: 127.96466573142229
Generation: 7, Best fitness: 127.96466573142229
Generation: 8, Best fitness: 127.96466573142229
Generation: 9, Best fitness: 127.96466573142229
Generation: 10, Best fitness: 127.96466573142229
Generation: 11, Best fitness: 127.96466573142229
Generation: 12, Best fitness: 127.96466573142229
Generation: 13, Best fitness: 127.96466573142229
Generation: 14, Best fitness: 127.96466573142229
Generation: 15, Best fitness: 127.96466573142229
Generation: 16, Best fitness: 124.666720032731
Generation: 17, Best fitness: 124.666720032731
Generation: 18, Best fitness: 124.666720032731
Generation: 19, Best fitness: 124.666720032731
Generation: 20, Best fitness: 124.6667

In [30]:
import simpy
import numpy as np

class Server:
    def __init__(self, env):
        self.env = env
        self.machine = simpy.Resource(env, capacity=1)
        self.latest_task_end_time = 0

    def process(self, task):
        with self.machine.request() as req:
            yield req
            yield self.env.timeout(task.duration)
            self.latest_task_end_time = self.env.now

class Task:
    def __init__(self, duration):
        self.duration = duration

class FitnessFunction:
    def __init__(self, tasks):
        self.tasks = tasks

    def __call__(self, solution):
        env = simpy.Environment()
        servers = [Server(env) for _ in range(10)]

        for task_duration, server_index in zip(self.tasks, solution):
            task = Task(task_duration)
            server = servers[int(server_index)]
            env.process(server.process(task))

        env.run()

        makespan = max(server.latest_task_end_time for server in servers)
        return makespan

class Flower:
    def __init__(self, decision_variables, lower_bound, upper_bound):
        self.position = np.random.uniform(low=lower_bound, high=upper_bound, size=decision_variables)
        self.fitness = float('inf')

class SimulatedAnnealing:
    def __init__(self, fitness_function, initial_solution, initial_temperature, cooling_rate):
        self.fitness_function = fitness_function
        self.current_solution = initial_solution
        self.current_fitness = self.fitness_function(self.current_solution)
        self.temperature = initial_temperature
        self.cooling_rate = cooling_rate

    def accept(self, candidate_fitness):
        if candidate_fitness < self.current_fitness:
            return True
        else:
            return np.random.rand() < np.exp((self.current_fitness - candidate_fitness) / self.temperature)

class FlowerPollination:
    def __init__(self, fitness_function, population_size=20, decision_variables=10, lower_bound=0, upper_bound=1, generations=100, switch_prob=0.8, gamma=0.1, beta=1.5):
        self.population = [Flower(decision_variables, lower_bound, upper_bound) for _ in range(population_size)]
        self.global_best = None
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.decision_variables = decision_variables
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.generations = generations
        self.switch_prob = switch_prob
        self.gamma = gamma
        self.beta = beta

    def levy_flight(self, beta):
        sigma = (np.math.gamma(1 + beta) * np.sin(np.pi * beta / 2) / (np.math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.normal(0, sigma, size=self.decision_variables)
        v = np.random.normal(0, 1, size=self.decision_variables)
        step = u / abs(v) ** (1 / beta)
        return step

    def global_pollination(self, flower):
        step_size = self.gamma * self.levy_flight(self.beta)
        new_position = flower.position + step_size * (flower.position - self.global_best.position)
        return new_position

    def local_pollination(self, flower):
        flower_j = np.random.choice(self.population)
        epsilon = np.random.uniform(low=0, high=1, size=self.decision_variables)
        new_position = flower.position + epsilon * (flower_j.position - flower.position)
        return new_position

    def pollination(self, flower):
        r = np.random.uniform(low=0, high=1)
        if r < self.switch_prob:
            new_position = self.global_pollination(flower)
        else:
            new_position = self.local_pollination(flower)
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def find_global_best(self):
        for flower in self.population:
            if flower.fitness < self.global_best.fitness:
                self.global_best = flower

    def optimize(self):
        self.global_best = self.population[0]
        self.global_best.fitness = self.fitness_function(self.global_best.position)
        sa = SimulatedAnnealing(self.fitness_function, self.global_best.position, initial_temperature=10, cooling_rate=0.9)

        for g in range(self.generations):
            for flower in self.population:
                flower.fitness = self.fitness_function(flower.position)
                self.find_global_best()

                new_position = self.pollination(flower)
                new_fitness = self.fitness_function(new_position)

                if sa.accept(new_fitness):
                    flower.position = new_position
                    flower.fitness = new_fitness

            self.find_global_best()
            print(f"Generation: {g}, Best fitness: {self.global_best.fitness}")

        return self.global_best

def generate_schedule(task_sizes):
    for task_size in task_sizes:
        task_durations = np.random.uniform(low=1, high=10, size=task_size)
        fitness_function = FitnessFunction(task_durations)

        fpa = FlowerPollination(fitness_function, population_size=100, decision_variables=task_size, lower_bound=0, upper_bound=9, generations=50)
        best_solution = fpa.optimize()
        print(f"Task size: {task_size}, Best makespan (Hybrid FPA-SA): {best_solution.fitness}")

task_sizes = [200]
generate_schedule(task_sizes)


Generation: 0, Best fitness: 132.5839104906782
Generation: 1, Best fitness: 130.4893766852978
Generation: 2, Best fitness: 126.64546995477377
Generation: 3, Best fitness: 126.64546995477377
Generation: 4, Best fitness: 126.64546995477377
Generation: 5, Best fitness: 129.10326079658228
Generation: 6, Best fitness: 129.10326079658228
Generation: 7, Best fitness: 129.10326079658228
Generation: 8, Best fitness: 129.10326079658228
Generation: 9, Best fitness: 129.10326079658228
Generation: 10, Best fitness: 129.10326079658228
Generation: 11, Best fitness: 129.10326079658228
Generation: 12, Best fitness: 129.10326079658228
Generation: 13, Best fitness: 129.10326079658228
Generation: 14, Best fitness: 129.10326079658228
Generation: 15, Best fitness: 129.10326079658228
Generation: 16, Best fitness: 129.10326079658228
Generation: 17, Best fitness: 129.10326079658228
Generation: 18, Best fitness: 129.10326079658228
Generation: 19, Best fitness: 129.10326079658228
Generation: 20, Best fitness: 13

In [51]:
import simpy
import numpy as np

class Server:
    def __init__(self, env, speed=1):
        self.env = env
        self.machine = simpy.Resource(env, capacity=1)
        self.latest_task_end_time = 0
        self.speed = speed
        self.expected_end_time = 0

    def process(self, task):
        with self.machine.request() as req:
            yield req
            yield self.env.timeout(task.duration / self.speed)
            self.latest_task_end_time = self.env.now
            self.expected_end_time = self.env.now + task.duration / self.speed


class Task:
    def __init__(self, duration):
        self.duration = duration

class FitnessFunction:
    def __init__(self, tasks, servers):
        self.tasks = tasks
        self.servers = servers

    def __call__(self, solution):
        env = simpy.Environment()
        servers = [Server(env, speed=speed) for speed in self.servers]

        for task_duration, server_index in zip(self.tasks, solution):
            task = Task(task_duration)
            server = servers[int(server_index)]
            env.process(server.process(task))

        env.run()

        makespan = max(server.latest_task_end_time for server in servers)
        return makespan

class Flower:
    def __init__(self, decision_variables, lower_bound, upper_bound):
        self.position = np.random.uniform(low=lower_bound, high=upper_bound, size=decision_variables)
        self.fitness = np.inf

class SimulatedAnnealing:
    def __init__(self, fitness_function, initial_solution, initial_temperature, cooling_rate):
        self.fitness_function = fitness_function
        self.current_solution = initial_solution
        self.current_fitness = self.fitness_function(self.current_solution)
        self.best_solution = self.current_solution
        self.best_fitness = self.current_fitness
        self.temperature = initial_temperature
        self.cooling_rate = cooling_rate

    def accept(self, candidate_fitness):
        if candidate_fitness < self.current_fitness:
            return True
        else:
            delta = self.current_fitness - candidate_fitness
            probability = np.exp(delta / self.temperature)
            return np.random.rand() < probability

    def cool_down(self):
        self.temperature *= self.cooling_rate

class FlowerPollination:
    def __init__(self, fitness_function, population_size=20, decision_variables=10, lower_bound=0, upper_bound=1, generations=100, switch_prob=0.8, gamma=0.1, beta=1.5):
        self.population = [Flower(decision_variables, lower_bound, upper_bound) for _ in range(population_size)]
        self.global_best = None
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.decision_variables = decision_variables
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.generations = generations
        self.switch_prob = switch_prob
        self.gamma = gamma
        self.beta = beta

    def levy_flight(self, beta):
        sigma = (np.math.gamma(1 + beta) * np.sin(np.pi * beta / 2) / (np.math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.normal(0, sigma, size=self.decision_variables)
        v = np.random.normal(0, 1, size=self.decision_variables)
        step = u / abs(v) ** (1 / beta)
        return step

    def global_pollination(self, flower):
        step_size = self.gamma * self.levy_flight(self.beta)
        new_position = flower.position + step_size * (flower.position - self.global_best.position)
        return new_position

    def local_pollination(self, flower):
        flower_j = np.random.choice(self.population)
        epsilon = np.random.uniform(low=0, high=1, size=self.decision_variables)
        new_position = flower.position + epsilon * (flower_j.position - flower.position)
        return new_position

    def pollination(self, flower):
        r = np.random.uniform(low=0, high=1)
        if r < self.switch_prob:
            new_position = self.global_pollination(flower)
        else:
            new_position = self.local_pollination(flower)
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def find_global_best(self):
        for flower in self.population:
            if flower.fitness < self.global_best.fitness:
                self.global_best = flower

    def optimize(self):
        self.global_best = self.population[0]
        self.global_best.fitness = self.fitness_function(self.global_best.position)
        sa = SimulatedAnnealing(self.fitness_function, self.global_best.position, initial_temperature=10, cooling_rate=0.9)

        for g in range(self.generations):
            for flower in self.population:
                flower.fitness = self.fitness_function(flower.position)
                self.find_global_best()

                new_position = self.pollination(flower)
                new_fitness = self.fitness_function(new_position)

                if sa.accept(new_fitness):
                    flower.position = new_position
                    flower.fitness = new_fitness

            sa.cool_down()  # cool down the temperature after each generation
            self.find_global_best()
            print(f"Generation: {g}, Best fitness: {self.global_best.fitness}")

        return self.global_best


class DynamicScheduler:
    def __init__(self, env, server_speeds):
        self.env = env
        self.servers = [Server(env, speed) for speed in server_speeds]

    def add_tasks(self, tasks):  # Modified method
        for task in tasks:
            self.add_task(task)

    def add_task(self, task):
        server = min(self.servers, key=lambda server: server.expected_end_time)
        self.env.process(server.process(task))

task_sizes = [200, 500, 1000]  # Your task sizes
tasks = [Task(size) for size in task_sizes]  # Create Task objects

env = simpy.Environment()
scheduler = DynamicScheduler(env, [1, 2, 3, 4, 5])  # Assuming 5 servers with speeds 1-5
scheduler.add_tasks(tasks)  # Add the tasks

env.run()


In [52]:
import simpy
import numpy as np

class Server:
    def __init__(self, env, speed=1):
        self.env = env
        self.machine = simpy.Resource(env, capacity=1)
        self.latest_task_end_time = 0
        self.speed = speed
        self.expected_end_time = 0

    def process(self, task):
        with self.machine.request() as req:
            yield req
            yield self.env.timeout(task.duration / self.speed)
            self.latest_task_end_time = self.env.now
            self.expected_end_time = self.env.now + task.duration / self.speed

class Task:
    def __init__(self, duration):
        self.duration = duration

class FitnessFunction:
    def __init__(self, tasks, servers):
        self.tasks = tasks
        self.servers = servers
        self.makespans = {task_duration: [] for task_duration in tasks}  # Store makespans for each task duration

    def __call__(self, solution):
        env = simpy.Environment()
        servers = [Server(env, speed=speed) for speed in self.servers]
        task_objects = []

        for task_duration, server_index in zip(self.tasks, solution):
            task = Task(task_duration)
            task_objects.append(task)  # Keep track of task objects
            server = servers[int(server_index)]
            env.process(server.process(task))

        env.run()

        # Store makespan for each task size
        for task in task_objects:
            self.makespans[task.duration].append(server.latest_task_end_time)

        makespan = max(server.latest_task_end_time for server in servers)
        return makespan

class Flower:
    def __init__(self, decision_variables, lower_bound, upper_bound):
        self.position = np.random.uniform(low=lower_bound, high=upper_bound, size=decision_variables)
        self.fitness = np.inf

class SimulatedAnnealing:
    def __init__(self, fitness_function, initial_solution, initial_temperature, cooling_rate):
        self.fitness_function = fitness_function
        self.current_solution = initial_solution
        self.current_fitness = self.fitness_function(self.current_solution)
        self.best_solution = self.current_solution
        self.best_fitness = self.current_fitness
        self.temperature = initial_temperature
        self.cooling_rate = cooling_rate

    def accept(self, candidate_fitness):
        if candidate_fitness < self.current_fitness:
            return True
        else:
            delta = self.current_fitness - candidate_fitness
            probability = np.exp(delta / self.temperature)
            return np.random.rand() < probability

    def cool_down(self):
        self.temperature *= self.cooling_rate

class FlowerPollination:
    def __init__(self, fitness_function, population_size=20, decision_variables=10, lower_bound=0, upper_bound=1, generations=100, switch_prob=0.8, gamma=0.1, beta=1.5):
        self.population = [Flower(decision_variables, lower_bound, upper_bound) for _ in range(population_size)]
        self.global_best = None
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.decision_variables = decision_variables
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.generations = generations
        self.switch_prob = switch_prob
        self.gamma = gamma
        self.beta = beta

    def levy_flight(self, beta):
        sigma = (np.math.gamma(1 + beta) * np.sin(np.pi * beta / 2) / (np.math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.normal(0, sigma, size=self.decision_variables)
        v = np.random.normal(0, 1, size=self.decision_variables)
        step = u / abs(v) ** (1 / beta)
        return step

    def global_pollination(self, flower):
        step_size = self.gamma * self.levy_flight(self.beta)
        new_position = flower.position + step_size * (flower.position - self.global_best.position)
        return new_position

    def local_pollination(self, flower):
        flower_j = np.random.choice(self.population)
        epsilon = np.random.uniform(low=0, high=1, size=self.decision_variables)
        new_position = flower.position + epsilon * (flower_j.position - flower.position)
        return new_position

    def pollination(self, flower):
        r = np.random.uniform(low=0, high=1)
        if r < self.switch_prob:
            new_position = self.global_pollination(flower)
        else:
            new_position = self.local_pollination(flower)
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def find_global_best(self):
        for flower in self.population:
            if flower.fitness < self.global_best.fitness:
                self.global_best = flower

    def optimize(self):
        self.global_best = self.population[0]
        self.global_best.fitness = self.fitness_function(self.global_best.position)
        sa = SimulatedAnnealing(self.fitness_function, self.global_best.position, initial_temperature=10, cooling_rate=0.9)

        for g in range(self.generations):
            for flower in self.population:
                flower.fitness = self.fitness_function(flower.position)
                self.find_global_best()

                new_position = self.pollination(flower)
                new_fitness = self.fitness_function(new_position)

                if sa.accept(new_fitness):
                    flower.position = new_position
                    flower.fitness = new_fitness

            sa.cool_down()  # cool down the temperature after each generation
            self.find_global_best()
            print(f"Generation: {g}, Best fitness: {self.global_best.fitness}")

        return self.global_best

# Optimization process
task_sizes = [200, 500, 1000]
servers = [1, 2, 3, 4, 5]

fitness_function = FitnessFunction(task_sizes, servers)
fp = FlowerPollination(fitness_function, decision_variables=len(task_sizes), lower_bound=0, upper_bound=len(servers)-1, generations=100)

best_flower = fp.optimize()

# Printing average makespan for each task size
for task_size, makespans in fitness_function.makespans.items():
    print(f"Task size: {task_size}, Average makespan: {np.mean(makespans)}")



Generation: 0, Best fitness: 250.0
Generation: 1, Best fitness: 250.0
Generation: 2, Best fitness: 250.0
Generation: 3, Best fitness: 250.0
Generation: 4, Best fitness: 250.0
Generation: 5, Best fitness: 250.0
Generation: 6, Best fitness: 250.0
Generation: 7, Best fitness: 250.0
Generation: 8, Best fitness: 250.0
Generation: 9, Best fitness: 250.0
Generation: 10, Best fitness: 250.0
Generation: 11, Best fitness: 250.0
Generation: 12, Best fitness: 250.0
Generation: 13, Best fitness: 250.0
Generation: 14, Best fitness: 250.0
Generation: 15, Best fitness: 250.0
Generation: 16, Best fitness: 250.0
Generation: 17, Best fitness: 250.0
Generation: 18, Best fitness: 250.0
Generation: 19, Best fitness: 250.0
Generation: 20, Best fitness: 250.0
Generation: 21, Best fitness: 200.0
Generation: 22, Best fitness: 200.0
Generation: 23, Best fitness: 200.0
Generation: 24, Best fitness: 200.0
Generation: 25, Best fitness: 200.0
Generation: 26, Best fitness: 200.0
Generation: 27, Best fitness: 200.0
Ge

In [1]:
import simpy
import numpy as np

class Server:
    def __init__(self, env, speed=1):
        self.env = env
        self.machine = simpy.Resource(env, capacity=1)
        self.latest_task_end_time = 0
        self.speed = speed
        self.expected_end_time = 0

    def process(self, task):
        with self.machine.request() as req:
            yield req
            yield self.env.timeout(task.duration / self.speed)
            self.latest_task_end_time = self.env.now
            self.expected_end_time = self.env.now + task.duration / self.speed

class Task:
    def __init__(self, duration):
        self.duration = duration

class FitnessFunction:
    def __init__(self, tasks, servers):
        self.tasks = tasks
        self.servers = servers
        self.makespans = {task_duration: [] for task_duration in tasks}  # Store makespans for each task duration

    def __call__(self, solution):
        env = simpy.Environment()
        servers = [Server(env, speed=speed) for speed in self.servers]
        task_objects = []

        for task_duration, server_index in zip(self.tasks, solution):
            task = Task(task_duration)
            task_objects.append(task)  # Keep track of task objects
            server = servers[int(server_index)]
            env.process(server.process(task))

        env.run()

        # Store makespan for each task size
        for task in task_objects:
            self.makespans[task.duration].append(server.latest_task_end_time)

        makespan = max(server.latest_task_end_time for server in servers)
        return makespan

class Flower:
    def __init__(self, decision_variables, lower_bound, upper_bound):
        self.position = np.random.uniform(low=lower_bound, high=upper_bound, size=decision_variables)
        self.fitness = np.inf

class SimulatedAnnealing:
    def __init__(self, fitness_function, initial_solution, initial_temperature, cooling_rate):
        self.fitness_function = fitness_function
        self.current_solution = initial_solution
        self.current_fitness = self.fitness_function(self.current_solution)
        self.best_solution = self.current_solution
        self.best_fitness = self.current_fitness
        self.temperature = initial_temperature
        self.cooling_rate = cooling_rate

    def accept(self, candidate_fitness):
        if candidate_fitness < self.current_fitness:
            return True
        else:
            delta = self.current_fitness - candidate_fitness
            probability = np.exp(delta / self.temperature)
            return np.random.rand() < probability

    def cool_down(self):
        self.temperature *= self.cooling_rate

class FlowerPollination:
    def __init__(self, fitness_function, population_size=20, decision_variables=10, lower_bound=0, upper_bound=1, generations=100, switch_prob=0.8, gamma=0.1, beta=1.5):
        self.population = [Flower(decision_variables, lower_bound, upper_bound) for _ in range(population_size)]
        self.global_best = None
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.decision_variables = decision_variables
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.generations = generations
        self.switch_prob = switch_prob
        self.gamma = gamma
        self.beta = beta

    def levy_flight(self, beta):
        sigma = (np.math.gamma(1 + beta) * np.sin(np.pi * beta / 2) / (np.math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.normal(0, sigma, size=self.decision_variables)
        v = np.random.normal(0, 1, size=self.decision_variables)
        step = u / abs(v) ** (1 / beta)
        return step

    def global_pollination(self, flower):
        step_size = self.gamma * self.levy_flight(self.beta)
        new_position = flower.position + step_size * (flower.position - self.global_best.position)
        return new_position

    def local_pollination(self, flower):
        flower_j = np.random.choice(self.population)
        epsilon = np.random.uniform(low=0, high=1, size=self.decision_variables)
        new_position = flower.position + epsilon * (flower_j.position - flower.position)
        return new_position

    def pollination(self, flower):
        r = np.random.uniform(low=0, high=1)
        if r < self.switch_prob:
            new_position = self.global_pollination(flower)
        else:
            new_position = self.local_pollination(flower)
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def find_global_best(self):
        for flower in self.population:
            if flower.fitness < self.global_best.fitness:
                self.global_best = flower

    def optimize(self):
        self.global_best = self.population[0]
        self.global_best.fitness = self.fitness_function(self.global_best.position)
        sa = SimulatedAnnealing(self.fitness_function, self.global_best.position, initial_temperature=10, cooling_rate=0.9)

        for g in range(self.generations):
            for flower in self.population:
                flower.fitness = self.fitness_function(flower.position)
                self.find_global_best()

                new_position = self.pollination(flower)
                new_fitness = self.fitness_function(new_position)

                if sa.accept(new_fitness):
                    flower.position = new_position
                    flower.fitness = new_fitness

            sa.cool_down()  # cool down the temperature after each generation
            self.find_global_best()
            print(f"Generation: {g}, Best fitness: {self.global_best.fitness}")

        return self.global_best

# Optimization process
servers = [1, 2, 3, 4, 5]
task_sizes = [200, 500, 1000]

for task_size in task_sizes:
    print(f"Optimizing for task size: {task_size}")
    fitness_function = FitnessFunction([task_size], servers)  # Single task size
    fp = FlowerPollination(fitness_function, decision_variables=1, lower_bound=0, upper_bound=len(servers)-1, generations=100)
    best_flower = fp.optimize()
    print(f"Average makespan for task size {task_size}: {np.mean(fitness_function.makespans[task_size])}")


Optimizing for task size: 200
Generation: 0, Best fitness: 40.0
Generation: 1, Best fitness: 40.0
Generation: 2, Best fitness: 40.0
Generation: 3, Best fitness: 40.0
Generation: 4, Best fitness: 40.0
Generation: 5, Best fitness: 40.0
Generation: 6, Best fitness: 40.0
Generation: 7, Best fitness: 40.0
Generation: 8, Best fitness: 40.0
Generation: 9, Best fitness: 40.0
Generation: 10, Best fitness: 50.0
Generation: 11, Best fitness: 40.0
Generation: 12, Best fitness: 40.0
Generation: 13, Best fitness: 40.0
Generation: 14, Best fitness: 40.0
Generation: 15, Best fitness: 40.0
Generation: 16, Best fitness: 40.0
Generation: 17, Best fitness: 40.0
Generation: 18, Best fitness: 40.0
Generation: 19, Best fitness: 40.0
Generation: 20, Best fitness: 40.0
Generation: 21, Best fitness: 40.0
Generation: 22, Best fitness: 40.0
Generation: 23, Best fitness: 40.0
Generation: 24, Best fitness: 40.0
Generation: 25, Best fitness: 40.0
Generation: 26, Best fitness: 40.0
Generation: 27, Best fitness: 40.0


In [3]:
import simpy
import numpy as np


class Server:
    def __init__(self, env, speed=1):
        self.env = env
        self.machine = simpy.Resource(env, capacity=1)
        self.latest_task_end_time = 0
        self.speed = speed
        self.expected_end_time = 0

    def process(self, task):
        with self.machine.request() as req:
            yield req
            yield self.env.timeout(task.duration / self.speed)
            self.latest_task_end_time = self.env.now
            self.expected_end_time = self.env.now + task.duration / self.speed


class Task:
    def __init__(self, duration):
        self.duration = duration


class FitnessFunction:
    def __init__(self, tasks, servers):
        self.tasks = tasks
        self.servers = servers
        # Store makespans for each task duration
        self.makespans = {task_duration: [] for task_duration in tasks}

    def __call__(self, solution):
        env = simpy.Environment()
        servers = [Server(env, speed=speed) for speed in self.servers]
        task_objects = []

        for task_duration, server_index in zip(self.tasks, solution):
            task = Task(task_duration)
            task_objects.append(task)  # Keep track of task objects
            server = servers[int(server_index)]
            env.process(server.process(task))

        env.run()

        # Store makespan for each task size
        for task in task_objects:
            self.makespans[task.duration].append(server.latest_task_end_time)

        makespan = max(server.latest_task_end_time for server in servers)
        return makespan


class Flower:
    def __init__(self, decision_variables, lower_bound, upper_bound):
        self.position = np.random.uniform(
            low=lower_bound, high=upper_bound, size=decision_variables)
        self.fitness = np.inf


class SimulatedAnnealing:
    def __init__(self, fitness_function, initial_solution, initial_temperature, final_temperature, cooling_rate):
        self.fitness_function = fitness_function
        self.current_solution = initial_solution
        self.current_fitness = self.fitness_function(self.current_solution)
        self.best_solution = self.current_solution
        self.best_fitness = self.current_fitness
        self.temperature = initial_temperature
        self.final_temperature = final_temperature
        self.cooling_rate = cooling_rate

    def accept(self, candidate_fitness):
        if candidate_fitness < self.current_fitness:
            return True
        else:
            delta = self.current_fitness - candidate_fitness
            probability = np.exp(delta / self.temperature)
            return np.random.rand() < probability

    def cool_down(self):
        if self.temperature > self.final_temperature:
            self.temperature *= self.cooling_rate


class FlowerPollination:
    def __init__(self, fitness_function, population_size=100, decision_variables=10, lower_bound=0, upper_bound=1, generations=1000, switch_prob=0.8, gamma=0.1, beta=1.5):
        self.population = [Flower(
            decision_variables, lower_bound, upper_bound) for _ in range(population_size)]
        self.global_best = None
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.decision_variables = decision_variables
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.generations = generations
        self.switch_prob = switch_prob
        self.gamma = gamma
        self.beta = beta

    def levy_flight(self, beta):
        sigma = (np.math.gamma(1 + beta) * np.sin(np.pi * beta / 2) /
                 (np.math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.normal(0, sigma, size=self.decision_variables)
        v = np.random.normal(0, 1, size=self.decision_variables)
        step = u / abs(v) ** (1 / beta)
        return step

    def global_pollination(self, flower):
        step_size = self.gamma * self.levy_flight(self.beta)
        new_position = flower.position + step_size * \
            (flower.position - self.global_best.position)
        return new_position

    def local_pollination(self, flower):
        flower_j = np.random.choice(self.population)
        epsilon = np.random.uniform(
            low=0, high=1, size=self.decision_variables)
        new_position = flower.position + epsilon * \
            (flower_j.position - flower.position)
        return new_position

    def pollination(self, flower):
        r = np.random.uniform(low=0, high=1)
        if r < self.switch_prob:
            new_position = self.global_pollination(flower)
        else:
            new_position = self.local_pollination(flower)
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def find_global_best(self):
        for flower in self.population:
            if flower.fitness < self.global_best.fitness:
                self.global_best = flower


    def optimize(self):
        self.global_best = self.population[0]
        self.global_best.fitness = self.fitness_function(self.global_best.position)
        sa = SimulatedAnnealing(self.fitness_function, self.global_best.position, initial_temperature=10, final_temperature=0.001, cooling_rate=0.9)

        for g in range(self.generations):
            for flower in self.population:
                flower.fitness = self.fitness_function(flower.position)
                self.find_global_best()

                new_position = self.pollination(flower)
                new_fitness = self.fitness_function(new_position)

                if sa.accept(new_fitness):
                    flower.position = new_position
                    flower.fitness = new_fitness

            sa.cool_down()  # cool down the temperature after each generation
            self.find_global_best()
            print(f"Generation: {g}, Best fitness: {self.global_best.fitness}")

        return self.global_best



# Optimization process
servers = [1, 2, 3, 4, 5]
task_sizes = [200, 500, 1000]

for task_size in task_sizes:
    print(f"Optimizing for task size: {task_size}")
    fitness_function = FitnessFunction(
        [task_size], servers)  # Single task size
    fp = FlowerPollination(fitness_function, decision_variables=1,
                           lower_bound=0, upper_bound=len(servers)-1, generations=1000)
    best_flower = fp.optimize()
    print(
        f"Average makespan for task size {task_size}: {np.mean(fitness_function.makespans[task_size])}")

Optimizing for task size: 200
Generation: 0, Best fitness: 50.0
Generation: 1, Best fitness: 50.0
Generation: 2, Best fitness: 50.0
Generation: 3, Best fitness: 50.0
Generation: 4, Best fitness: 40.0
Generation: 5, Best fitness: 40.0
Generation: 6, Best fitness: 40.0
Generation: 7, Best fitness: 40.0
Generation: 8, Best fitness: 40.0
Generation: 9, Best fitness: 40.0
Generation: 10, Best fitness: 40.0
Generation: 11, Best fitness: 40.0
Generation: 12, Best fitness: 40.0
Generation: 13, Best fitness: 40.0
Generation: 14, Best fitness: 40.0
Generation: 15, Best fitness: 40.0
Generation: 16, Best fitness: 40.0
Generation: 17, Best fitness: 40.0
Generation: 18, Best fitness: 40.0
Generation: 19, Best fitness: 40.0
Generation: 20, Best fitness: 40.0
Generation: 21, Best fitness: 40.0
Generation: 22, Best fitness: 40.0
Generation: 23, Best fitness: 40.0
Generation: 24, Best fitness: 40.0
Generation: 25, Best fitness: 40.0
Generation: 26, Best fitness: 40.0
Generation: 27, Best fitness: 40.0


In [ ]:
import simpy
import numpy as np

class Server:
    def __init__(self, env, speed=1, latency=0):
        self.env = env
        self.machine = simpy.Resource(env, capacity=1)
        self.latest_task_end_time = 0
        self.speed = speed
        self.latency = latency  # New attribute for latency
        self.expected_end_time = 0

    def process(self, task):
        with self.machine.request() as req:
            yield req
            yield self.env.timeout((task.duration / self.speed) + self.latency)  # Add latency to processing time
            self.latest_task_end_time = self.env.now
            self.expected_end_time = self.env.now + (task.duration / self.speed) + self.latency  # Add latency to expected end time

class Task:
    def __init__(self, duration):
        self.duration = duration

class FitnessFunction:
    def __init__(self, tasks, servers):
        self.tasks = tasks
        self.servers = servers
        self.makespans = {task_duration: [] for task_duration in tasks}  # Store makespans for each task duration

    def __call__(self, solution):
        env = simpy.Environment()
        servers = [Server(env, speed=speed, latency=latency) for speed, latency in self.servers]  # Initialize servers with speed and latency
        task_objects = []

        for task_duration, server_index in zip(self.tasks, solution):
            task = Task(task_duration)
            task_objects.append(task)  # Keep track of task objects
            server = servers[int(server_index)]
            env.process(server.process(task))

        env.run()

        # Store makespan for each task size
        for task in task_objects:
            self.makespans[task.duration].append(server.latest_task_end_time)

        makespan = max(server.latest_task_end_time for server in servers)
        return makespan

class Flower:
    def __init__(self, decision_variables, lower_bound, upper_bound):
        self.position = np.random.uniform(
            low=lower_bound, high=upper_bound, size=decision_variables)
        self.fitness = np.inf


class SimulatedAnnealing:
    def __init__(self, fitness_function, initial_solution, initial_temperature, final_temperature, cooling_rate):
        self.fitness_function = fitness_function
        self.current_solution = initial_solution
        self.current_fitness = self.fitness_function(self.current_solution)
        self.best_solution = self.current_solution
        self.best_fitness = self.current_fitness
        self.temperature = initial_temperature
        self.final_temperature = final_temperature
        self.cooling_rate = cooling_rate

    def accept(self, candidate_fitness):
        if candidate_fitness < self.current_fitness:
            return True
        else:
            delta = self.current_fitness - candidate_fitness
            probability = np.exp(delta / self.temperature)
            return np.random.rand() < probability

    def cool_down(self):
        if self.temperature > self.final_temperature:
            self.temperature *= self.cooling_rate


class FlowerPollination:
    def __init__(self, fitness_function, population_size=100, decision_variables=10, lower_bound=0, upper_bound=1, generations=1000, switch_prob=0.8, gamma=0.1, beta=1.5):
        self.population = [Flower(
            decision_variables, lower_bound, upper_bound) for _ in range(population_size)]
        self.global_best = None
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.decision_variables = decision_variables
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.generations = generations
        self.switch_prob = switch_prob
        self.gamma = gamma
        self.beta = beta

    def levy_flight(self, beta):
        sigma = (np.math.gamma(1 + beta) * np.sin(np.pi * beta / 2) /
                 (np.math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.normal(0, sigma, size=self.decision_variables)
        v = np.random.normal(0, 1, size=self.decision_variables)
        step = u / abs(v) ** (1 / beta)
        return step

    def global_pollination(self, flower):
        step_size = self.gamma * self.levy_flight(self.beta)
        new_position = flower.position + step_size * \
            (flower.position - self.global_best.position)
        return new_position

    def local_pollination(self, flower):
        flower_j = np.random.choice(self.population)
        epsilon = np.random.uniform(
            low=0, high=1, size=self.decision_variables)
        new_position = flower.position + epsilon * \
            (flower_j.position - flower.position)
        return new_position

    def pollination(self, flower):
        r = np.random.uniform(low=0, high=1)
        if r < self.switch_prob:
            new_position = self.global_pollination(flower)
        else:
            new_position = self.local_pollination(flower)
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def find_global_best(self):
        for flower in self.population:
            if flower.fitness < self.global_best.fitness:
                self.global_best = flower


    def optimize(self):
        self.global_best = self.population[0]
        self.global_best.fitness = self.fitness_function(self.global_best.position)
        sa = SimulatedAnnealing(self.fitness_function, self.global_best.position, initial_temperature=10, final_temperature=0.001, cooling_rate=0.9)

        for g in range(self.generations):
            for flower in self.population:
                flower.fitness = self.fitness_function(flower.position)
                self.find_global_best()

                new_position = self.pollination(flower)
                new_fitness = self.fitness_function(new_position)

                if sa.accept(new_fitness):
                    flower.position = new_position
                    flower.fitness = new_fitness

            sa.cool_down()  # cool down the temperature after each generation
            self.find_global_best()
            print(f"Generation: {g}, Best fitness: {self.global_best.fitness}")

        return self.global_best

# Then when initializing servers, specify both speed and latency.
cloud_servers = [(1, 0.1), (2, 0.1), (3, 0.1), (4, 0.1), (5, 0.1)]  # For cloud servers
edge_servers = [(1, 0.01), (2, 0.01), (3, 0.01), (4, 0.01), (5, 0.01)]  # For edge servers

# Run your optimization for both cloud and edge servers.
for task_size in task_sizes:
    print(f"Optimizing for cloud servers with task size: {task_size}")
    fitness_function = FitnessFunction([task_size], cloud_servers)  # Single task size
    fp = FlowerPollination(fitness_function, decision_variables=1, lower_bound=0, upper_bound=len(servers)-1, generations=100)
    best_flower = fp.optimize()
    print(f"Average makespan for cloud servers with task size {task_size}: {np.mean(fitness_function.makespans[task_size])}")

    print(f"Optimizing for edge servers with task size: {task_size}")
    fitness_function = FitnessFunction([task_size], edge_servers)  # Single task size
    fp = FlowerPollination(fitness_function, decision_variables=1, lower_bound=0, upper_bound=len(servers)-1, generations=100)
    best_flower = fp.optimize()
    print(f"Average makespan for edge servers with task size {task_size}: {np.mean(fitness_function.makespans[task_size])}")
